In [11]:
from yo_fluq import *
from bs4 import BeautifulSoup
import re
from talents.german.quiz import content as c

def parse_words(file):
    with open(file, encoding="utf-8") as f:
        html = f.read()
    
    soup = BeautifulSoup(html, "html.parser")
    
    words = []
    
    for a in soup.find_all("a", href=True):
        href = a["href"]
        # Проверка, что это ссылка на dwds.de/wb/
        if re.match(r"^https?://www\.dwds\.de/wb/", href):
            for sup in a.find_all("sup"):
                sup.decompose()
            word = a.get_text(strip=True)
            words.append(word)

    return words

def parse_files():
    words = []
    seen_words = set()
    for level in ['A1', 'A2', 'B1']:
        for word in parse_words(f"source_files/{level}.html"):
            if word in seen_words:
                continue
            seen_words.add(word)
            words.append(c.Word(level, word))
    return words

words = parse_files()

In [12]:
from yo_fluq import *

wik = c.Wiktionary('de')
for word in Query.en(words).feed(fluq.with_progress_bar()):
    word.wiktionary = wik.get(word.word)

  0%|          | 0/3289 [00:00<?, ?it/s]

In [13]:

for word in words:
    if word.wiktionary is not None:
        word.grammar = c.NounGrammar.parse_wiktionary(word.wiktionary)

nouns = [w for w in words if w.grammar is not None and w.grammar.grammar_error() is None]

In [14]:
comp = c.SimilarityComputer()
table = comp.compute_table([w.word for w in nouns])

In [15]:
similarities = comp.table_to_list(table)

In [8]:
for w in nouns:
    w.similar_words = tuple(similarities[w.word])

In [9]:
import os
os.makedirs('temp', exist_ok=True)
FileIO.write_pickle(nouns, 'temp/words.pkl')

In [10]:
w

Word(level='B1', word='Übung', wiktionary=None, grammar=NounGrammar(genus=('f',), plural=('Übungen',), singular=('Übung',)), similar_words=('Training', 'Praktikant', 'Praktikum', 'Praktikantin', 'Lehrling'))